In [1]:
import pandas as pd
import datetime
import numpy as np
from mido import Message, MidiFile, MidiTrack, MetaMessage
from collections import namedtuple

## Data processing: 

In [2]:
df = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/nationalcasedeath/csv')
df

,country,country_code,continent,population,indicator,weekly_count,year_week,rate_14_day,cumulative_count,source
0,Afghanistan,AFG,Asia,38928341,cases,0,2020-01,NaN,0,"Epidemic intelligence, national weekly data"
1,Afghanistan,AFG,Asia,38928341,cases,0,2020-02,0.000000,0,"Epidemic intelligence, national weekly data"
2,Afghanistan,AFG,Asia,38928341,cases,0,2020-03,0.000000,0,"Epidemic intelligence, national weekly data"
3,Afghanistan,AFG,Asia,38928341,cases,0,2020-04,0.000000,0,"Epidemic intelligence, national weekly data"
4,Afghanistan,AFG,Asia,38928341,cases,0,2020-05,0.000000,0,"Epidemic intelligence, national weekly data"
...,...,...,...,...,...,...,...,...,...,...
23287,Zimbabwe,ZWE,Africa,14862927,deaths,229,2021-04,35.053661,1234,"Epidemic intelligence, national weekly data"
23288,Zimbabwe,ZWE,Africa,14862927,deaths,92,2021-05,21.597361,1326,"Epidemic intelligence, national weekly data"
23289,Zimbabwe,ZWE,Africa,14862927,deaths,74,2021-06,11.168729,1400,"Epidemic intelligence, national weekly data"
23290,Zimbabwe,ZWE,Africa,14862927,deaths,41,2021-07,7.737372,1441,"Epidemic intelligence, national weekly data"


In [3]:
df['date'] = pd.to_datetime([datetime.datetime.strptime(d + '-1', '%G-%V-%u') for d in df['year_week']])

In [4]:
df_cases = df.loc[df['indicator'] == 'cases', ['country', 'country_code', 'date', 'weekly_count', 'cumulative_count']].rename(columns = {
    'weekly_count': 'weekly_cases', 
    'cumulative_count': 'cumulative_cases'
})

In [5]:
df_deaths = df.loc[df['indicator'] == 'deaths', ['country', 'country_code', 'date', 'weekly_count', 'cumulative_count']].rename(columns = {
    'weekly_count': 'weekly_deaths', 
    'cumulative_count': 'cumulative_deaths'
})

In [67]:
df_weekly = pd.merge(df_cases, df_deaths, on=['country', 'country_code', 'date'])
df_weekly

,country,country_code,date,weekly_cases,cumulative_cases,weekly_deaths,cumulative_deaths
0,Afghanistan,AFG,2019-12-30,0,0,0,0
1,Afghanistan,AFG,2020-01-06,0,0,0,0
2,Afghanistan,AFG,2020-01-13,0,0,0,0
3,Afghanistan,AFG,2020-01-20,0,0,0,0
4,Afghanistan,AFG,2020-01-27,0,0,0,0
...,...,...,...,...,...,...,...
11641,Zimbabwe,ZWE,2021-01-25,2228,33548,229,1234
11642,Zimbabwe,ZWE,2021-02-01,1004,34552,92,1326
11643,Zimbabwe,ZWE,2021-02-08,620,35172,74,1400
11644,Zimbabwe,ZWE,2021-02-15,690,35862,41,1441


In [7]:
continent_names_df = pd.read_csv(
    '~/Dropbox (BBC)/Visual Journalism/Data/2020/vjdata.26866.global.covid.interactive/\
derived/bbc_world_v2_4326_centroid_region.csv'
)

continent_df = pd.merge(df_weekly, 
         continent_names_df[['iso3', 'bbc-region-code']], 
         left_on = 'country_code', 
         right_on = 'iso3').groupby(
    ['date', 'bbc-region-code']
)[['weekly_cases', 'weekly_deaths']].sum().reset_index()

def rename_continent(bbc_region): 
    switcher = {
        'region_as': 'Asia',
        'region_af': 'Africa',
        'region_am_na': 'North America',
        'region_am_lac': 'Latin America',
        'region_me': 'Middle East',
        'region_eu': 'Europe',
        'region_oc': 'Oceania'
    }
    return switcher[bbc_region]

continent_df['continent_name'] = continent_df['bbc-region-code'].map(rename_continent)
continent_df = continent_df[['date', 'continent_name', 'weekly_cases', 'weekly_deaths']]

continent_df = continent_df.sort_values(by = ['continent_name', 'date']).reset_index(drop=True)
# continent_df['cases_rolling_average'] = continent_df.groupby(
#     'continent_name'
# ).rolling(7)['cases'].mean().reset_index(drop=True)
# continent_df['deaths_rolling_average'] = continent_df.groupby(
#     'continent_name'
# ).rolling(7)['deaths'].mean().reset_index(drop=True)
continent_df['cumulative_deaths'] = continent_df.groupby(['continent_name'])['weekly_deaths'].cumsum()

In [8]:
continent_df

,date,continent_name,weekly_cases,weekly_deaths,cumulative_deaths
0,2019-12-30,Africa,0,0,0
1,2020-01-06,Africa,0,0,0
2,2020-01-13,Africa,0,0,0
3,2020-01-20,Africa,0,0,0
4,2020-01-27,Africa,0,0,0
...,...,...,...,...,...
422,2021-01-25,Oceania,571,5,1207
423,2021-02-01,Oceania,248,3,1210
424,2021-02-08,Oceania,262,3,1213
425,2021-02-15,Oceania,189,3,1216


In [69]:
df_weekly.loc[-df_weekly['country'].str.contains('total')]

,country,country_code,date,weekly_cases,cumulative_cases,weekly_deaths,cumulative_deaths
0,Afghanistan,AFG,2019-12-30,0,0,0,0
1,Afghanistan,AFG,2020-01-06,0,0,0,0
2,Afghanistan,AFG,2020-01-13,0,0,0,0
3,Afghanistan,AFG,2020-01-20,0,0,0,0
4,Afghanistan,AFG,2020-01-27,0,0,0,0
...,...,...,...,...,...,...,...
11641,Zimbabwe,ZWE,2021-01-25,2228,33548,229,1234
11642,Zimbabwe,ZWE,2021-02-01,1004,34552,92,1326
11643,Zimbabwe,ZWE,2021-02-08,620,35172,74,1400
11644,Zimbabwe,ZWE,2021-02-15,690,35862,41,1441


In [84]:
df_weekly['year_month'] = df_weekly.date.dt.strftime('%Y-%m')

In [85]:
global_df = df_weekly.loc[-df_weekly['country'].str.contains('total')].groupby(
    ['year_month']
)[['weekly_cases', 'weekly_deaths']].sum().reset_index()
global_df['cumulative_deaths'] = global_df['weekly_deaths'].cumsum()

In [86]:
global_df

,year_month,weekly_cases,weekly_deaths,cumulative_deaths
0,2019-12,59,0,0
1,2020-01,17314,362,362
2,2020-02,71081,2678,3040
3,2020-03,1187230,68333,71373
4,2020-04,2208184,176332,247705
5,2020-05,2667630,123257,370962
6,2020-06,5312916,159398,530360
7,2020-07,6693067,154028,684388
8,2020-08,9333127,204918,889306
9,2020-09,8291797,148860,1038166


In [79]:
for part in phrase_3.parts: 
    total_duration = 0
    for note in part.notes: 
        total_duration += note.duration
    print(total_duration)

4.0
4.0
4.0
4.0


In [25]:
0.5 * 0.75

0.375

In [87]:
Note = namedtuple('Note', ['pitch', 'duration', 'accent'])
Note.__new__.__defaults__ = (None,) * len(Note._fields)
Part = namedtuple('Part', 'notes')
Phrase = namedtuple('Phrase', 'parts')

phrase_1 = Phrase([Part([Note(58, 0.5), Note(58, 0.375), Note(58, 0.125), Note(58, 1), 
                         Note(58, 0.5), Note(58, 0.375), Note(58, 0.125), Note(58, 1)]),
                   Part([Note(53, 0.5), Note(54, 0.5), Note(53, 0.5), Note(54, 0.5), 
                         Note(53, 0.5), Note(54, 0.5), Note(53, 0.5), Note(54, 0.5)]), 
                   Part([Note(41, 0.5), Note(42, 0.5), Note(41, 0.5), Note(42, 0.5), 
                         Note(41, 0.5), Note(42, 0.5), Note(41, 0.5), Note(42, 0.5)]), 
                   Part([Note(34, 0.5), Note(37, 0.5), Note(34, 0.5), Note(37, 0.5), 
                         Note(34, 0.5), Note(37, 0.5), Note(34, 0.5), Note(37, 0.5)])])

phrase_2 = Phrase([Part([Note(58, 0.5), Note(58, 0.375), Note(58, 0.125), Note(58, 0.5), 
                         Note(61, 0.375), Note(60, 0.125), Note(60, 0.375), Note(58, 0.125), 
                         Note(58, 0.375), Note(58, 0.125), Note(58, 1)]),
                   Part([Note(53, 0.5), Note(54, 0.5), Note(53, 0.5), Note(54, 0.5), 
                         Note(53, 0.5), Note(54, 0.5), Note(53, 0.5), Note(54, 0.5)]), 
                   Part([Note(41, 0.5), Note(42, 0.5), Note(41, 0.5), Note(42, 0.5), 
                         Note(41, 0.5), Note(42, 0.5), Note(41, 0.5), Note(42, 0.5)]), 
                   Part([Note(34, 0.5), Note(37, 0.5), Note(34, 0.5), Note(37, 0.5), 
                         Note(34, 0.5), Note(37, 0.5), Note(34, 0.5), Note(37, 0.5)])])

phrase_3 = Phrase([Part([Note(61, 0.5), Note(61, 0.375), Note(61, 0.125), Note(61, 0.5), 
                         Note(65, 0.375), Note(63, 0.125), Note(63, 0.375), Note(61, 0.125), 
                         Note(61, 0.375), Note(61, 0.125), Note(61, 1)]),
                   Part([Note(58, 0.5), Note(58, 0.5), Note(58, 0.5), 
                         Note(61, 0.375), Note(60, 0.125), Note(60, 0.375), Note(58, 0.125), 
                         Note(58, 0.375), Note(58, 0.125), Note(58, 1)]),
                   Part([Note(41, 0.5), Note(42, 0.5), Note(41, 0.5), Note(42, 0.5), 
                         Note(41, 0.5), Note(42, 0.5), Note(41, 0.5), Note(42, 0.5)]), 
                   Part([Note(34, 0.5), Note(37, 0.5), Note(34, 0.5), Note(37, 0.5), 
                         Note(34, 0.5), Note(37, 0.5), Note(34, 0.5), Note(37, 0.5)])])


phrase_4 = Phrase([Part([Note(70, 0.25), Note(68, 0.25), Note(66, 0.375), Note(65, 0.125), Note(65, 1), 
                         Note(70, 0.25), Note(68, 0.25), Note(66, 0.375), Note(65, 0.125), Note(65, 1)]),
                   Part([Note(61, 0.5), Note(61, 0.5), Note(61, 0.5), Note(61, 0.5), 
                         Note(61, 0.5), Note(61, 0.5), Note(61, 0.5), Note(61, 0.5)]),
                   Part([Note(41, 0.5), Note(42, 0.5), Note(41, 0.5), Note(42, 0.5), 
                         Note(41, 0.5), Note(42, 0.5), Note(41, 0.5), Note(42, 0.5)]), 
                   Part([Note(34, 0.5), Note(37, 0.5), Note(34, 0.5), Note(37, 0.5), 
                         Note(34, 0.5), Note(37, 0.5), Note(34, 0.5), Note(37, 0.5)])])

# The maximum cases value will be used to calibrate the volume of each track:
max_global_weekly_deaths_value = global_df['weekly_deaths'].max()
max_cumulative_deaths_value = global_df['cumulative_deaths'].max()

# 41 48 55 62 55 48
# Reference for instrument values, general midi: https://en.wikipedia.org/wiki/General_MIDI
Continent = namedtuple('Continent', ['name',          'part_idx','octave_shift', 'pan', 'instrument', 'channel'])
asia =                  Continent(   'Asia',          3,          0,             120,   42,           0) #cello
# middle_east =           Continent(   'Middle East',   1,          0,             100,   73,           1) #flute
africa =                Continent(   'Africa',        2,          0,              80,   41,           2) #viola
europe =                Continent(   'Europe',        0,          0,              63,   71,           3) #clnet
latin_america =         Continent(   'Latin America', 1,          0,              40,   40,           4) #vln
north_america =         Continent(   'North America', 0,          0,              10,   40,           5) #vln

continents = [asia, africa, europe, latin_america, north_america]

class Sequence(): 
    def __init__(self, part, volume, octave_shift, is_first_day, offset): 
        self.notes = part.notes.copy()
        self.volume = volume
        self.octave_shift = octave_shift
        self.is_first_day = is_first_day
        self.offset = offset
        self.beat_duration = 960 * 2
        
    def add_to_track(self, track):
        if self.is_first_day: 
            delay = int(self.offset * self.beat_duration - self.beat_duration)
            if delay < 0: 
                delay = 0
            # Override: 
            delay = 0
        else: 
            delay = 0
        note = self.notes.pop(0)
        if note.accent in ['accent', 'stac']: 
            velocity = 120
        else: 
            velocity = 64
        track.append(Message('note_on', note = note.pitch, velocity = velocity, time = delay))
        track.append(Message('control_change', control=7, value=self.volume, time=0))
        while(len(self.notes) > 0): 
            if note.accent == 'stac': 
                delay = int(note.duration * self.beat_duration / 2)
                track.append(Message('note_off', note = note.pitch, time = delay))
                track.append(Message('note_off', note = note.pitch, time = delay))
            else: 
                delay = int(note.duration * self.beat_duration)
                track.append(Message('note_off', note = note.pitch, time = delay))
            note = self.notes.pop(0)
            if note.accent in ['accent', 'stac']: 
                velocity = 120
            else: 
                velocity = 64
            track.append(Message('note_on', note = note.pitch, velocity = velocity, time = 0))
        delay = note.duration * self.beat_duration
        track.append(Message('note_off', note = note.pitch, time = int(delay)))
        

mid = MidiFile()
for continent in continents:
    name = continent.name
    part_idx = continent.part_idx
    channel = continent.channel
    
    print(continent)
    filtered_df = continent_df.loc[continent_df['continent_name'] == continent.name]
    filtered_df = global_df
    weekly_deaths_data = filtered_df['weekly_deaths'].apply(lambda x: 0 if np.isnan(x) else int(x))
    cumulative_deaths_data = filtered_df['cumulative_deaths'].apply(lambda x: 0 if np.isnan(x) else int(x))
#     deaths_data = continent_df.groupby('date')['cumulative_deaths'].sum().values
    track = MidiTrack()
    mid.tracks.append(track)
    # Set up track:
    track.append(MetaMessage('track_name', name=name, time=0))
    track.append(MetaMessage('time_signature', numerator=5, denominator=4, time=0)) # does not work
    # Instrumentation:
    track.append(Message('program_change', channel=channel, program=continent.instrument, time=0))
    # Pan:
    track.append(Message('control_change', channel=channel, control=10, value=continent.pan, time=0))
    
    max_continent_weekly_deaths_value = weekly_deaths_data.max()
    weekly_deaths_threshold_width = max_continent_weekly_deaths_value / 4
    
    counter = 0
    for weekly_deaths, cumulative_deaths in zip(weekly_deaths_data, cumulative_deaths_data): 
        if cumulative_deaths > 0: 
            volume = int(cumulative_deaths / max_cumulative_deaths_value * 80) + 40
        else: 
            volume = 0
        is_first_day = counter == 0
        counter += 1
        
        if weekly_deaths <= weekly_deaths_threshold_width: 
            phrase = phrase_1
        elif weekly_deaths <= weekly_deaths_threshold_width * 2: 
            phrase = phrase_2
        elif weekly_deaths <= weekly_deaths_threshold_width * 3: 
            phrase = phrase_3
        elif weekly_deaths <= weekly_deaths_threshold_width * 4: 
            phrase = phrase_4            
        
        part = phrase.parts[part_idx]
        
        sequence = Sequence(part, volume, continent.octave_shift, is_first_day, channel)
        sequence.add_to_track(track)
        
        
    
mid.save('./output/v9_test_002.mid')

Continent(name='Asia', part_idx=3, octave_shift=0, pan=120, instrument=42, channel=0)
Continent(name='Africa', part_idx=2, octave_shift=0, pan=80, instrument=41, channel=2)
Continent(name='Europe', part_idx=0, octave_shift=0, pan=63, instrument=71, channel=3)
Continent(name='Latin America', part_idx=1, octave_shift=0, pan=40, instrument=40, channel=4)
Continent(name='North America', part_idx=0, octave_shift=0, pan=10, instrument=40, channel=5)


In [32]:
continent_df

,date,continent_name,weekly_cases,weekly_deaths,cumulative_deaths
0,2019-12-30,Africa,0,0,0
1,2020-01-06,Africa,0,0,0
2,2020-01-13,Africa,0,0,0
3,2020-01-20,Africa,0,0,0
4,2020-01-27,Africa,0,0,0
...,...,...,...,...,...
422,2021-01-25,Oceania,571,5,1207
423,2021-02-01,Oceania,248,3,1210
424,2021-02-08,Oceania,262,3,1213
425,2021-02-15,Oceania,189,3,1216


In [179]:
continent_df.groupby('continent_name').describe()

cases                                                       \
                count          mean            std   min      25%      50%   
continent_name                                                               
Africa          331.0   7126.927492    5908.213204   0.0    806.0   7409.0   
Asia            331.0  40428.755287   34867.652514  99.0   4298.5  37881.0   
Europe          331.0  71114.616314  106552.599562   0.0  15824.0  24629.0   
Latin America   331.0  43951.075529   33375.645846   0.0   5504.0  47628.0   
Middle East     331.0  11595.069486    8114.153073   0.0   4177.0  12175.0   
North America   331.0  54255.619335   57949.862046   0.0  21730.0  37030.0   
Oceania         331.0    142.854985     184.700323   0.0     12.0     52.0   

                                   deaths               ...  \
                    75%        max  count         mean  ...   
continent_name                                          ...   
Africa          11509.0    21460.0  331.0   155.178248  ...   
Asia            67588.0   111867.0  331.0   633.595166  ...   
Europe          69055.0  1070501.0  331.0  1540.450151  ...   
Latin America   71816.5   116035.0  331.0  1457.138973  ...   
Middle East     16340.5    31085.0  331.0   280.462236  ...   
North America   61060.5   256409.0  331.0   989.858006  ...   
Oceania           229.5     1404.0  331.0     3.160121  ...   

               deaths_rolling_average              cumulative_deaths  \
                                  75%          max             count   
continent_name                                                         
Africa                     251.000000   404.285714             331.0   
Asia                      1023.428571  1466.428571             331.0   
Europe                    2436.714286  5326.714286             331.0   
Latin America             2294.428571  3153.428571             331.0   
Middle East                417.428571   665.857143             331.0   
North America             1223.857143  2759.571429             331.0   
Oceania                      3.285714    22.142857             331.0   

                                                                      \
                         mean            std  min      25%       50%   
continent_name                                                         
Africa           15608.737160   16282.777724  0.0    714.0    8226.0   
Asia             69788.652568   70765.267507  1.0   5305.0   39157.0   
Europe          179810.012085  123283.452948  0.0  89893.0  196506.0   
Latin America   175461.510574  168213.326588  0.0   3482.0  129858.0   
Middle East      30042.942598   28314.425635  0.0   5281.0   21605.0   
North America   134773.226586   96834.455975  0.0  34071.5  139603.0   
Oceania            397.903323     407.977795  0.0     71.5     128.0   

                                    
                     75%       max  
continent_name                      
Africa           29270.5   51364.0  
Asia            135054.0  209720.0  
Europe          230049.5  509889.0  
Latin America   336951.5  482313.0  
Middle East      50300.5   92833.0  
North America   214240.0  327643.0  
Oceania            913.5    1046.0  

[7 rows x 40 columns]

In [159]:
phrase_1

Phrase(parts=[Part(notes=[Note(pitch=69, duration=4), Note(pitch=69, duration=1)]), Part(notes=[Note(pitch=62, duration=4), Note(pitch=61, duration=1)]), Part(notes=[Note(pitch=53, duration=4), Note(pitch=52, duration=1)]), Part(notes=[])])

In [57]:
for msg in mid.tracks[0]:
    print(msg)

<meta message track_name name='Asia' time=0>
<meta message time_signature numerator=6 denominator=8 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
program_change channel=0 program=42 time=0
control_change channel=0 control=10 value=120 time=0
note_on channel=0 note=50 velocity=64 time=0
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 val

control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
note_off channel=0 note=55 velocity=0 time=0
control_change channel=0 control=7 value=0 time=0
note_on channel=0 note=51 velocity=64 time=0
control_change channel=0 control=7 value=6 time=12
control_change channel=0 control=7 value=10 time=12
control_change channel=0 control=7 value=14 time=12
control_change channel=0 control=7 value=18 time=12
control_change channel=0 control=7 value=22 time=12
control_change channel=0 control=7 value=26 time=12
control_change channel=0 control=7 value=30 time=12
control_change channel=0 control=7 value=34 time=12
control_change channel=0 control=7 value=38 time=12
control_change channel=0 control=7 value=42 time=12
control_change channel=0 control=7 value=42 time=12
control_change channel=0 control=7

control_change channel=0 control=7 value=0 time=12
note_off channel=0 note=50 velocity=0 time=0
control_change channel=0 control=7 value=0 time=0
note_on channel=0 note=52 velocity=64 time=0
control_change channel=0 control=7 value=6 time=12
control_change channel=0 control=7 value=10 time=12
control_change channel=0 control=7 value=14 time=12
control_change channel=0 control=7 value=18 time=12
control_change channel=0 control=7 value=22 time=12
control_change channel=0 control=7 value=26 time=12
control_change channel=0 control=7 value=30 time=12
control_change channel=0 control=7 value=34 time=12
control_change channel=0 control=7 value=38 time=12
control_change channel=0 control=7 value=42 time=12
control_change channel=0 control=7 value=42 time=12
control_change channel=0 control=7 value=38 time=12
control_change channel=0 control=7 value=34 time=12
control_change channel=0 control=7 value=30 time=12
control_change channel=0 control=7 value=26 time=12
control_change channel=0 contr

note_on channel=0 note=53 velocity=64 time=0
control_change channel=0 control=7 value=9 time=12
control_change channel=0 control=7 value=13 time=12
control_change channel=0 control=7 value=17 time=12
control_change channel=0 control=7 value=21 time=12
control_change channel=0 control=7 value=25 time=12
control_change channel=0 control=7 value=29 time=12
control_change channel=0 control=7 value=33 time=12
control_change channel=0 control=7 value=37 time=12
control_change channel=0 control=7 value=41 time=12
control_change channel=0 control=7 value=45 time=12
control_change channel=0 control=7 value=45 time=12
control_change channel=0 control=7 value=41 time=12
control_change channel=0 control=7 value=37 time=12
control_change channel=0 control=7 value=33 time=12
control_change channel=0 control=7 value=29 time=12
control_change channel=0 control=7 value=25 time=12
control_change channel=0 control=7 value=21 time=12
control_change channel=0 control=7 value=17 time=12
control_change chann

control_change channel=0 control=7 value=27 time=12
control_change channel=0 control=7 value=31 time=12
control_change channel=0 control=7 value=35 time=12
control_change channel=0 control=7 value=39 time=12
control_change channel=0 control=7 value=43 time=12
control_change channel=0 control=7 value=47 time=12
control_change channel=0 control=7 value=51 time=12
control_change channel=0 control=7 value=51 time=12
control_change channel=0 control=7 value=47 time=12
control_change channel=0 control=7 value=43 time=12
control_change channel=0 control=7 value=39 time=12
control_change channel=0 control=7 value=35 time=12
control_change channel=0 control=7 value=31 time=12
control_change channel=0 control=7 value=27 time=12
control_change channel=0 control=7 value=23 time=12
control_change channel=0 control=7 value=19 time=12
control_change channel=0 control=7 value=15 time=12
control_change channel=0 control=7 value=15 time=1080
control_change channel=0 control=7 value=11 time=12
control_ch

control_change channel=0 control=7 value=50 time=12
control_change channel=0 control=7 value=54 time=12
control_change channel=0 control=7 value=58 time=12
control_change channel=0 control=7 value=62 time=12
control_change channel=0 control=7 value=62 time=12
control_change channel=0 control=7 value=58 time=12
control_change channel=0 control=7 value=54 time=12
control_change channel=0 control=7 value=50 time=12
control_change channel=0 control=7 value=46 time=12
control_change channel=0 control=7 value=42 time=12
control_change channel=0 control=7 value=38 time=12
control_change channel=0 control=7 value=34 time=12
control_change channel=0 control=7 value=30 time=12
control_change channel=0 control=7 value=26 time=12
control_change channel=0 control=7 value=26 time=1080
control_change channel=0 control=7 value=22 time=12
control_change channel=0 control=7 value=18 time=12
control_change channel=0 control=7 value=14 time=12
control_change channel=0 control=7 value=10 time=12
control_ch

control_change channel=0 control=7 value=68 time=12
control_change channel=0 control=7 value=64 time=12
control_change channel=0 control=7 value=60 time=12
control_change channel=0 control=7 value=56 time=12
control_change channel=0 control=7 value=52 time=12
control_change channel=0 control=7 value=48 time=12
control_change channel=0 control=7 value=44 time=12
control_change channel=0 control=7 value=40 time=12
control_change channel=0 control=7 value=36 time=12
control_change channel=0 control=7 value=32 time=12
control_change channel=0 control=7 value=32 time=1080
control_change channel=0 control=7 value=28 time=12
control_change channel=0 control=7 value=24 time=12
control_change channel=0 control=7 value=20 time=12
control_change channel=0 control=7 value=16 time=12
control_change channel=0 control=7 value=12 time=12
control_change channel=0 control=7 value=8 time=12
control_change channel=0 control=7 value=4 time=12
control_change channel=0 control=7 value=0 time=12
control_chang

control_change channel=0 control=7 value=46 time=12
control_change channel=0 control=7 value=42 time=12
control_change channel=0 control=7 value=38 time=12
control_change channel=0 control=7 value=34 time=12
control_change channel=0 control=7 value=30 time=12
control_change channel=0 control=7 value=26 time=12
control_change channel=0 control=7 value=22 time=12
control_change channel=0 control=7 value=22 time=1080
control_change channel=0 control=7 value=18 time=12
control_change channel=0 control=7 value=14 time=12
control_change channel=0 control=7 value=10 time=12
control_change channel=0 control=7 value=6 time=12
control_change channel=0 control=7 value=2 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
note_off channel=0 note=53 velocity=0 time=0
control_change channel=0